In [4]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex







In [12]:
import os

# Create data folder if not exists
if not os.path.exists("data"):
    os.makedirs("data")
    print("📁 Created folder: data")

# Create a sample text file
sample_path = "data/test.txt"
with open(sample_path, "w", encoding="utf-8") as f:
    f.write("This is a test document for LlamaIndex. It contains sample text used for indexing and querying.")

print(f"📝 Created sample file: {sample_path}")


📝 Created sample file: data/test.txt


In [ ]:
import tensorflow as tf
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import time
from tensorflow.keras import layers

# ==========================================
# 1. Configuration & Data Loading
# ==========================================
print(f"TensorFlow Version: {tf.__version__}")

BUFFER_SIZE = 60000
BATCH_SIZE = 256
EPOCHS = 50
NOISE_DIM = 100
num_examples_to_generate = 16

# We will use the seed to visualize progress on the same noise vector over time
seed = tf.random.normal([num_examples_to_generate, NOISE_DIM])

def load_data():
    """Loads and preprocesses the MNIST dataset."""
    (train_images, _), (_, _) = tf.keras.datasets.mnist.load_data()
    
    # Normalize the images to [-1, 1]
    train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
    train_images = (train_images - 127.5) / 127.5 

    # Batch and shuffle the data
    train_dataset = tf.data.Dataset.from_tensor_slices(train_images)
    train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
    return train_dataset

# ==========================================
# 2. Model Definitions
# ==========================================

def make_generator_model():
    """
    Creates the Generator model.
    Input: Random Noise (seed)
    Output: 28x28x1 Image (Grayscale)
    """
    model = tf.keras.Sequential()
    
    # Foundation for 7x7 image
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(NOISE_DIM,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((7, 7, 256)))
    assert model.output_shape == (None, 7, 7, 256)

    # Upsample to 14x14
    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, 7, 7, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Upsample to 14x14
    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 14, 14, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Upsample to 28x28
    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 28, 28, 1)

    return model

def make_discriminator_model():
    """
    Creates the Discriminator model.
    Input: Image
    Output: Probability (Real vs Fake)
    """
    model = tf.keras.Sequential()
    
    # Downsample
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[28, 28, 1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    # Downsample
    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1)) # Output logits

    return model

# ==========================================
# 3. Loss & Optimizers
# ==========================================

cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    """Calculates how well the discriminator distinguishes real from fake."""
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

def generator_loss(fake_output):
    """Calculates how well the generator fooled the discriminator."""
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator = make_generator_model()
discriminator = make_discriminator_model()

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

# ==========================================
# 4. Custom Training Loop
# ==========================================

@tf.function # Compiles the function for performance
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, NOISE_DIM])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        # 1. Generate fake images
        generated_images = generator(noise, training=True)

        # 2. Discriminate real and fake images
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        # 3. Calculate Loss
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    # 4. Calculate Gradients
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    # 5. Apply Gradients (Backprop)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    
    return gen_loss, disc_loss

def train(dataset, epochs):
    print("Starting training...")
    for epoch in range(epochs):
        start = time.time()
        
        gen_loss_avg = 0
        disc_loss_avg = 0
        batch_count = 0

        for image_batch in dataset:
            g_loss, d_loss = train_step(image_batch)
            gen_loss_avg += g_loss
            disc_loss_avg += d_loss
            batch_count += 1

        # Produce images for the GIF (or just save them)
        # generate_and_save_images(generator, epoch + 1, seed)

        print(f'Time for epoch {epoch + 1} is {time.time()-start:.2f} sec - Gen Loss: {gen_loss_avg/batch_count:.4f} - Disc Loss: {disc_loss_avg/batch_count:.4f}')

    # Generate after the final epoch
    generate_and_save_images(generator, epochs, seed)

def generate_and_save_images(model, epoch, test_input):
    # Notice `training` is set to False.
    # This is so all layers run in inference mode (batchnorm).
    predictions = model(test_input, training=False)

    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        # Rescale back to [0, 255] for display
        plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
        plt.axis('off')

    print(f"Saving image for epoch {epoch}...")
    # In a real environment, you would uncomment this:
    # plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
    # plt.show()
    plt.close()

# ==========================================
# 5. Main Execution
# ==========================================

if __name__ == "__main__":
    try:
        dataset = load_data()
        print("Data loaded. Training DCGAN...")
        train(dataset, EPOCHS)
        print("Training Complete.")
    except Exception as e:
        print(f"An error occurred: {e}")



TensorFlow Version: 2.20.0


c:\Users\azanw\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\azanw\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Data loaded. Training DCGAN...
Starting training...
Time for epoch 1 is 215.34 sec - Gen Loss: 0.7946 - Disc Loss: 1.0528
Time for epoch 2 is 206.36 sec - Gen Loss: 0.8170 - Disc Loss: 1.2879
Time for epoch 3 is 187.44 sec - Gen Loss: 0.8633 - Disc Loss: 1.1816
Time for epoch 4 is 166.64 sec - Gen Loss: 0.8960 - Disc Loss: 1.2118
Time for epoch 5 is 165.14 sec - Gen Loss: 0.9117 - Disc Loss: 1.2705
Time for epoch 6 is 166.49 sec - Gen Loss: 0.8700 - Disc Loss: 1.2966
Time for epoch 7 is 166.81 sec - Gen Loss: 0.8432 - Disc Loss: 1.2807
Time for epoch 8 is 166.34 sec - Gen Loss: 0.9172 - Disc Loss: 1.2011
Time for epoch 9 is 167.94 sec - Gen Loss: 0.8687 - Disc Loss: 1.2914
Time for epoch 10 is 166.42 sec - Gen Loss: 0.9145 - Disc Loss: 1.2200
Time for epoch 11 is 163.19 sec - Gen Loss: 0.9108 - Disc Loss: 1.2107
Time for epoch 12 is 162.62 sec - Gen Loss: 0.9316 - Disc Loss: 1.2311
Time for epoch 13 is 163.92 sec - Gen Loss: 1.1048 - Disc Loss: 1.1008
Time for epoch 14 is 165.18 sec - 